#### HeatMap on a dinamic map

In [ ]:
# Vectorized Haversine (meters)
def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0  # Earth radius (m)
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return 2*R*np.arcsin(np.sqrt(a))

# Your junction & threshold
junction_lat, junction_lon = 32.075412, 34.775316
threshold_m = 20.0

# Compute distances vectorized
dist_m = haversine_m(events_trips["lat"].to_numpy(),
                     events_trips["lon"].to_numpy(),
                     junction_lat, junction_lon)

# Filter rows near the junction
near_mask = dist_m <= threshold_m
selected_trips_df = events_trips.loc[near_mask].copy()

print(f"Found {near_mask.sum()} rows near the junction (≤ {threshold_m} m).")

In [ ]:
selected_trips_df.columns

#### Heatmap Matrix by date

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ----- inputs (same as before) -----
DF       = selected_trips_df
TIME_COL = "event_time"
LAT_COL  = "lat"
LON_COL  = "lon"

targets = [
    {"name":"Point Dizingof center", "lat":32.075412, "lon":34.775316}
]

RADIUS_METERS = 120
COUNT_UNIQUE_ID = False
ID_COL = "vehicle_id"

# ----- helpers -----
def haversine_np(lat1, lon1, lat2, lon2):
    R = 6371000.0
    lat1 = np.radians(lat1.astype(float)); lon1 = np.radians(lon1.astype(float))
    lat2 = np.radians(lat2);              lon2 = np.radians(lon2)
    dlat = lat2 - lat1; dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

# ----- 1) prep: assign each row to nearest target (within radius) -----
df = DF.copy()
df["date"] = pd.to_datetime(df[TIME_COL], errors="coerce").dt.date
df = df.dropna(subset=[LAT_COL, LON_COL, "date"]).copy()

lat_arr = df[LAT_COL].astype(float).to_numpy()
lon_arr = df[LON_COL].astype(float).to_numpy()

target_names = [t["name"] for t in targets]
t_lats = np.array([t["lat"] for t in targets], dtype=float)
t_lons = np.array([t["lon"] for t in targets], dtype=float)

dists        = haversine_np(lat_arr[:,None], lon_arr[:,None], t_lats[None,:], t_lons[None,:])
nearest_idx  = dists.argmin(axis=1)
nearest_dist = dists.min(axis=1)

df["target_name"] = np.where(nearest_dist <= RADIUS_METERS,
                             np.array(target_names, dtype=object)[nearest_idx],
                             np.nan)
df = df[df["target_name"].notna()].copy()

# ----- 2) build 3 × (#days) matrix -----
if COUNT_UNIQUE_ID:
    grouped = (df.groupby(["target_name","date"], as_index=False)[ID_COL]
                 .nunique().rename(columns={ID_COL:"count"}))
else:
    grouped = (df.groupby(["target_name","date"], as_index=False)
                 .size().rename(columns={"size":"count"}))

# All days that appear, sorted
all_days = pd.Series(grouped["date"].unique()).sort_values().tolist()

usage_by_day = (grouped
    .pivot(index="target_name", columns="date", values="count")
    .reindex(index=target_names)                # keep row order
    .reindex(columns=all_days, fill_value=0)    # ensure all days
    .fillna(0).astype(int)
)

# ----- 3) plot heatmap with clear x labels + save PNG -----
DATE_FMT = "%Y-%m-%d"   # change to "%d %b" or "%a\n%d %b" if you prefer
TARGET_VISIBLE_TICKS = 31  # ~how many x labels to show

values = usage_by_day.values
n_days = usage_by_day.shape[1]
step = max(1, int(np.ceil(n_days / TARGET_VISIBLE_TICKS)))  # show every k-th date

fig, ax = plt.subplots(figsize=(max(12, n_days*0.45), 4))
im = ax.imshow(values, aspect="auto")

# x ticks (every 'step' day)
xticks = np.arange(n_days)
ax.set_xticks(xticks[::step])
ax.set_xticklabels([pd.to_datetime(d).strftime(DATE_FMT) for i, d in enumerate(usage_by_day.columns) if i % step == 0],
                   rotation=45, ha="right")

# y ticks
ax.set_yticks(np.arange(usage_by_day.shape[0]))
ax.set_yticklabels(usage_by_day.index.tolist())

ax.set_title("Trips per Target Point by Day")
ax.set_xlabel("Day")
ax.set_ylabel("Target Point")

# annotate each cell with its value (optional—comment out if too dense)
for (i, j), val in np.ndenumerate(values):
    if j % step == 0:  # annotate only on shown ticks to reduce clutter
        ax.text(j, i, int(val), ha="center", va="center", fontsize=8)

# light grid (optional)
ax.set_xticks(np.arange(-.5, n_days, 1), minor=True)
ax.set_yticks(np.arange(-.5, usage_by_day.shape[0], 1), minor=True)
ax.grid(which="minor", linestyle="-", linewidth=0.5, alpha=0.25)
ax.tick_params(which="minor", bottom=False, left=False)

cbar = plt.colorbar(im, ax=ax)
cbar.set_label("Number of Trips", rotation=90)

plt.tight_layout()
out_path = "trips_per_target_by_day.png"
plt.savefig(out_path, dpi=200, bbox_inches="tight")
plt.show()

print(f"Saved to: {out_path}")


### VALIDATION MAP

In [ ]:
parking_spots_df = pd.read_csv(r'C:\Users\aviyaoh\Downloads\csv_and_merge_files\Parking spots\English GIS initial parking spots.csv') #GIS file
arking_spot_name = "Spot 1"  
spot = Parking_Spots_df[Parking_Spots_df['spot_name'] == parking_spot_name].iloc[0]
center_lon = spot['X']
center_lat = spot['Y']

In [ ]:
import folium

# 1. Pick the parking spot (by name from Parking_Spots_df)
parking_spot_name = "Spot 1"  # <-- תחליפי לשם שקיים בעמודת spot_name שלך

spot = Parking_Spots_df[Parking_Spots_df['spot_name'] == parking_spot_name].iloc[0]
center_lon = spot['X']
center_lat = spot['Y']

# 2. Create a folium map centered on the selected parking spot
m = folium.Map(location=[center_lat, center_lon],
               zoom_start=18,
               tiles='OpenStreetMap')

# 3. Add a 90 m radius circle around the parking spot
folium.Circle(
    location=[center_lat, center_lon],
    radius=90,                # meters
    color='orange',
    weight=2,
    fill=False,
    popup=f"Parking spot: {parking_spot_name}"
).add_to(m)

# 4. Add trip start (green) and trip end (red) points from DF

starts = DF[DF['event_types'] == 'trip_start']
ends   = DF[DF['event_types'] == 'trip_end']

for _, row in starts.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=2,
        color='green',
        fill=True,
        fill_opacity=0.6,
        opacity=0.6
    ).add_to(m)

for _, row in ends.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=2,
        color='red',
        fill=True,
        fill_opacity=0.6,
        opacity=0.6
    ).add_to(m)

# 5. Show the map (Jupyter/Colab)
m

In [ ]:
# Filter starts and ends from DF
starts = DF[DF['event_types'] == 'trip_start']
ends   = DF[DF['event_types'] == 'trip_end']

# Optional: אם יש לך המון נקודות, אפשר לסנן רק אלה שקרובות יחסית למרכז (לשיפור ביצועים)
# למשל רק נקודות בתוך ריבוע קטן סביב החניון:
# DEG_DELTA = 0.002  # בערך ~200 מ'
# starts = starts[(starts['lat'].between(center_lat - DEG_DELTA, center_lat + DEG_DELTA)) &
#                 (starts['lon'].between(center_lon - DEG_DELTA, center_lon + DEG_DELTA))]
# ends   = ends[(ends['lat'].between(center_lat - DEG_DELTA, center_lat + DEG_DELTA)) &
#               (ends['lon'].between(center_lon - DEG_DELTA, center_lon + DEG_DELTA))]

# Add trip start points – green
for _, row in starts.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=2,
        color='green',
        fill=True,
        fill_opacity=0.6,
        opacity=0.6
    ).add_to(m)

# Add trip end points – red
for _, row in ends.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=2,
        color='red',
        fill=True,
        fill_opacity=0.6,
        opacity=0.6
    ).add_to(m)

### Date-Time Cycle - Rapid Graph ---- An importent one!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# 1) הכנת הנתונים
df['event_time'] = pd.to_datetime(df['event_time'], errors='coerce')


# חלון שבועיים ו-trip_start בלבד
end   = df['event_time'].max()
start = end - pd.Timedelta(days=10)
mask  = (df['event_time'] >= start) & (df['event_time'] <= end)
df2   = df.loc[mask & (df['event_types'] == 'trip_start')].copy()



# אגרגציה לשעה + מילוי שעות חסרות ב-0
full_idx = pd.date_range(start=start.floor('H'), end=end.ceil('H'), freq='H')

hourly = (
    df2.set_index('event_time')
       .resample('H').size()                      # ספירה לשעה
       .reindex(full_idx, fill_value=0)           # כל השעות בחלון, כולל 0
       .rename_axis('event_time')
       .reset_index(name='count')
)

# 2) ציור
fig, ax = plt.subplots(figsize=(14, 4.5))

# הקו השחור
ax.plot(hourly['event_time'], hourly['count'], color='black', linewidth=1.6, label='trip_start')
ax.set_ylabel('Trips per hour')
ax.set_title('Trip starts – hourly over 10 days - Dizingof Center')

import matplotlib.dates as mdates

# --- מחליף את בלוק ה-major/minor הישן ---



ax.xaxis.set_minor_locator(mdates.HourLocator(byhour=[0, 9, 17]))
ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M'))
ax.tick_params(axis='x', which='major', labelsize=10, pad=6)

ax_dates = ax.secondary_xaxis('bottom')
ax_dates.xaxis.set_major_locator(mdates.DayLocator())
ax_dates.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax_dates.tick_params(axis='x', pad=22, labelsize=9)
# אל תתני לציר הראשי להציג תאריכים (רק קווי יום בלי טקסט)
ax.xaxis.set_major_locator(mdates.DayLocator())

ax.tick_params(axis='x', which='major', labelbottom=False)




# קווי חצות להפרדה ויזואלית
for day in pd.date_range(hourly['event_time'].min().normalize(),
                         hourly['event_time'].max().normalize() + pd.Timedelta(days=1),
                         freq='D'):
    ax.axvline(day, linestyle='--', linewidth=0.8, alpha=0.25)

# גבולות החלון (שבועיים)
ax.set_xlim([start, end])

# רשת עדינה
ax.grid(axis='x', which='major', alpha=0.12)
ax.grid(axis='y', alpha=0.25)
ax.legend(loc='upper right', frameon=False)

plt.tight_layout()
plt.show()
